# Training chemCPA

## Download data

This notebook has two dependencies:

sciplex3
wget 
rdkit embeddings

This notebook is a self-contained way to train the chemCPA model.

In [1]:
import sys
import os
sys.path.append('..')

import wandb
wandb.login()

from chemCPA.data import DataModule
from chemCPA.model import ComPert
import torch
import numpy as np
import time
import yaml
import lightning as L
from lightning.pytorch import seed_everything
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
%load_ext autoreload
%autoreload 2


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: martin-biroscak (biroscak). Use `wandb login --relogin` to force relogin


## load configuration

In [2]:
with open('../experiments/hydra_config/dataset/sciplex_lincs.yaml') as file:
    config_dataset = yaml.safe_load(file)
with open('../experiments/hydra_config/model/cpa.yaml') as file:
    config_model = yaml.safe_load(file)
with open('../experiments/hydra_config/test/test.yaml') as file:
    config_test = yaml.safe_load(file)
with open('../experiments/hydra_config/train/train.yaml') as file:
    config_train = yaml.safe_load(file)

#seed_everything()

## initialize the dataset module

In [3]:
dm = DataModule(config_model['hparams']['batch_size'],
                config_train['full_eval_during_train'],
                **config_dataset['data_params'])

In [4]:
print(len(dm.datasets['training']))
print(len(dm.datasets['training_control']))
print(len(dm.datasets['training_treated']))
print(len(dm.datasets['test']))
print(len(dm.datasets['test_control']))
print(len(dm.datasets['test_treated']))
print(len(dm.datasets['ood']))

248258
9105
239153
52966
1963
51003
53416


In [5]:
print(dm.datasets['training'].num_covariates)
print(dm.datasets['training'].num_drugs)
print(dm.datasets['training'].num_knockouts)

[3, 17]
188
0


In [26]:
dm.datasets["ood"].dosages

array([tensor([0.]), tensor([10000.]), tensor([1000.]), ...,
       tensor([1000.]), tensor([1000.]), tensor([100.])], dtype=object)

## initialize the model

In [14]:
# initialize the model
# the knockouts embeddings are initialized as random 
model = ComPert(
    dm.datasets['training'].num_genes,
    dm.datasets['training'].num_drugs,
    dm.datasets['training'].num_knockouts,
    dm.datasets['training'].num_covariates,
    config_model['hparams'],
    config_train,
    config_test,
    **config_model['additional_params'],
    drug_embedding_dimension=dm.datasets['training'].drug_embedding_dimension,
    knockout_embedding_dimension=dm.datasets['training'].knockout_embedding_dimension,
    train_adversarial=False
)
    

False


## Initialize the trainer

In [23]:
early_stop_callback = EarlyStopping('average_r2_score', 
                                    patience=model.hparams.training_params['patience'], 
                                    mode='max')
wandb_logger = WandbLogger(
        project = config_model['model_type'] + "_" + config_dataset['dataset_type'],
        save_dir = config_model['save_dir']     
    )
inference_mode = ((not config_train['run_eval_disentangle']) and (not config_test['run_eval_disentangle']))
trainer = L.Trainer(
    logger=wandb_logger,
    max_epochs=config_train['num_epochs'],
    max_time=config_train['max_time'],
    check_val_every_n_epoch= config_train['checkpoint_freq'],
    default_root_dir=config_model['save_dir'],
    profiler="advanced",
    callbacks=[early_stop_callback],
    inference_mode=inference_mode,
    num_sanity_val_steps=1,
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## train the model

In [ ]:
trainer.fit(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type            | Params
-----------------------------------------------------------
0 | loss_autoencoder       | GaussianNLLLoss | 0     
1 | encoder                | MLP             | 1.4 M 
2 | decoder                | MLP             | 1.9 M 
3 | adversary_drugs        | MLP             | 74.6 K
4 | drug_embedding_encoder | MLP             | 693 K 
5 | loss_adversary_drugs   | CELoss          | 0     
6 | dosers                 | ModuleList      | 36.3 K
-----------------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.095    Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


## test the model

In [27]:
trainer.test(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Testing: |                                                                                                        | 0/? [00:00<?, ?it/s]

## reload the model from check point

In [ ]:
#reload the model from check point
#model = ComPert.load_from_checkpoint('train_data/CPA/3gm2eppz/checkpoints/epoch=14-step=10560.ckpt')

## drawing the evaluation results

In [ ]:
from plotnine import *
from chemCPA.train import evaluate_logfold_r2, evaluate_r2, evaluate_r2_sc
import pandas as pd

In [ ]:
#draw the logfold r2
def draw_logfold_r2(autoencoder, ds_treated, ds_ctrl):
    logfold_score, signs_score = evaluate_logfold_r2(autoencoder, ds_treated, ds_ctrl, return_mean=False)
    df = pd.DataFrame(
        data = {'logfold_score': logfold_score, 'signs_score': signs_score}
    )
    df = pd.melt(df, value_vars=['logfold_score', 'signs_score'], var_name='score_type', value_name='score')
    p = ggplot(df, aes(x='factor(score_type)', y='score', fill='factor(score_type)')) + geom_boxplot() + scale_y_continuous(limits=(-1,1))
    return p

In [ ]:
#draw the r2
def draw_r2(autoencoder, dataset, genes_control):
    mean_score, mean_score_de, var_score, var_score_de = evaluate_r2(autoencoder, dataset, genes_control, return_mean=False)
    df = pd.DataFrame(
        data = {'mean_score': mean_score, 
                'mean_score_de': mean_score_de,
                'var_score': var_score,
                'var_score_de':var_score_de
                }
    )
    df = pd.melt(df, value_vars=['mean_score', 'mean_score_de', 'var_score', 'var_score_de'], 
                 var_name='score_type', value_name='score')
    p = ggplot(df, aes(x='factor(score_type)', y='score', fill='factor(score_type)')) + geom_boxplot()+ scale_y_continuous(limits=(0,1))
    return p

In [ ]:
#draw the r2 sc
def draw_r2_sc(autoencoder, dataset):
    mean_score, mean_score_de, var_score, var_score_de = evaluate_r2_sc(autoencoder, dataset, return_mean=False)
    df = pd.DataFrame(
        data = {'mean_score': mean_score, 
                'mean_score_de': mean_score_de,
                'var_score': var_score,
                'var_score_de':var_score_de
                }
    )
    df = pd.melt(df, value_vars=['mean_score', 'mean_score_de', 'var_score', 'var_score_de'], 
                 var_name='score_type', value_name='score')
    p = ggplot(df, aes(x='factor(score_type)', y='score', fill='factor(score_type)')) + geom_boxplot()+ scale_y_continuous(limits=(0,1))
    return p